<a href="https://colab.research.google.com/github/Lebengeniesser/Pytorch_Practice/blob/master/%EC%98%88%EC%A0%9C%EB%A1%9C_%EB%B0%B0%EC%9A%B0%EB%8A%94_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#<PyTorch: NumPy>
import numpy as np
import math

#무작위로 입력과 출력 데이터를 생성
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)
#무작위로 가중치를 초기화 함
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6 # 10^-6
for t in range(2000):
  #순전파 단계: 예측값 y를 계산
  #y = a + bx +c x^2 + d x^3
  y_pred = a + b *x + c *x**2 + d*x**3
  #손실(loss)을 계산하고 출력함
  loss = np.square(y_pred - y).sum()
  if t % 100 == 99:
    print(t, loss)

  #손실에 따른 a,b,c,d의 변화도를 계산하고 역전파 한다.
  grad_y_pred = 2.0 * (y_pred - y)  
  grad_a = grad_y_pred.sum()
  grad_b = (grad_y_pred * x).sum()
  grad_c = (grad_y_pred * x**2).sum()
  grad_d = (grad_y_pred * x**3).sum()

  a -= learning_rate *grad_a
  b -= learning_rate *grad_b
  c -= learning_rate *grad_c
  d -= learning_rate *grad_d

print(f"Result: y = {a}+{b}x+{c}x^2+{d}x^3")

#<PyTorch: Tensor>
import torch
import math
dtype = torch.float
device = torch.device("cpu")
#무작위로 입력과 출력 데이터를 생성
x = torch.linspace(-math.pi, math.pi, 2000, device = device, dtype = dtype)
y = torch.sin(x)

#무작위로 가주잋를 초기화
a = torch.randn((), device = device, dtype = dtype)
b = torch.randn((), device = device, dtype = dtype)
c = torch.randn((), device = device, dtype = dtype)
d = torch.randn((), device = device, dtype = dtype)

learning_rate = 1e-6
for t in range(2000):
  y_pred = a + b*x + c*x**2 + d*x**3
  loss = (y_pred - y).pow(2).sum().item()
  if t % 100 == 99:
    print(t, loss)
  grad_y_pred = 2.0 * (y_pred - y)  
  grad_a = grad_y_pred.sum()
  grad_b = (grad_y_pred * x).sum()
  grad_c = (grad_y_pred * x**2).sum()
  grad_d = (grad_y_pred * x**3).sum()

  a -= learning_rate *grad_a
  b -= learning_rate *grad_b
  c -= learning_rate *grad_c
  d -= learning_rate *grad_d

print(f"Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3")
#NumPy 와 Tensor의 연산을 비교하였을 때 Tensor가 손실이 더 적었다.

#Autograd 신경망의 순전파 단계에서 연산그래프를 정의하게 됨.
#이 그래프의 노드는 텐서이고, 엣지는 입력텐서로 부터 출력텐서르 만들어내는 함수(노드=텐서, 엣지=출력)
#  y = a + bx + cx^2+dx^3 대신 y = a + bP3(c+dx)모델을 정의함.
#P3(x) = 1/2 (5x^3 -3x)은 3차 르장드르다항식이다. P3의 순전파와 역전파 연산을 위한 새로운 autograd Function를 작성하고, 이를 사용하여 모델을 구현

#PyTorch: nn (인공신경망)
#Tensorflow는 Keras, TensorFlow-Slim, TFLearn같은 패키지들이 연산 그래프를 고수준으로 추상화 하여 제공하므로 신경망을 구축하는데 유용함

import torch
import math
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

#이예제는 선형함수이므로, 선형 계층 신경망으로 간주
p = torch.tensor([1,2,3])
xx = x.unsqueeze(-1).pow(p) # pow(y,x) y^x라는 뜻 허수면 에러
print(p)
print(xx)

model = torch.nn.Sequential(
    torch.nn.Linear(3,1),
    torch.nn.Flatten(0,1)
)
loss_fn = torch.nn.MSELoss(reduction="sum")

learning_rate = 1e-6
for t in range(2000):
  y_pred = model(xx)
  loss = loss_fn(y_pred, y)
  if t % 100 == 99:
    print(t, loss.item())

  #역전파 단계를 실행하기 전에 변화도를 0으로 만든다.
  model.zero_grad()  

  #역전파 단계: 모델의 학습 가능한 모든 매개변수에 대해 손실의 변화도를 계산한다.
  #내보적으로 각 Module의 매개변수는 requires_grad=True일 때 텐서에 저장되므로,
  # 아래 호출은 모델의 모든 학습 가능한 매개변수의 변화도를 계산하게 된다.
  loss.backward()

  #경사하강법을 사용하여 가중치를 갱신한다.
  #각 매개변수는 텐서이므로, 이전에 했던 것처럼 변화도에 접근할 수 있다.
  with torch.no_grad():
    for param in model.parameters():
      param -= learning_rate * param.grad
linear_layer = model[0]      

#선형 계층에서, 매개변수는 'weights'와 'bias'로 저장된다.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()}x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

#NumPy, Tensor, Autograd확인 결과 Autograd의 손실이 더 적었다.

#PyTorch: optim패키지는 최적화 알고리즘에 대한 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체를 제공한다.
#지금까지 nn패키지를 사용하여 모델을 저으이하지만, 모델을 최적화할 때는 optim패키지가 제공하는 RMSProp알고리즘을 사용한다.





99 3290.1234906665286
199 2330.841853639788
299 1652.0071822688828
399 1171.6272102047465
499 831.6844419257623
599 591.122591506794
699 420.88802963668485
799 300.4208481764237
899 215.1717506942424
999 154.84487180231974
1099 112.15430225375201
1199 81.94413956559254
1299 60.56578868203662
1399 45.4373065037672
1499 34.73157039503099
1599 27.155609439406796
1699 21.79444734800987
1799 18.000597279184376
1899 15.315862406799988
1999 13.415997776366625
Result: y = 0.07178095819699351+0.8565921549070911x+-0.012383409729281494x^2+-0.09330923944367465x^3
99 2582.837890625
199 1815.859130859375
299 1278.0068359375
399 900.6360473632812
499 635.7311401367188
599 449.6878662109375
699 318.9712829589844
799 227.0890655517578
899 162.47799682617188
999 117.02677917480469
1099 85.04228210449219
1199 62.526878356933594
1299 46.67210006713867
1399 35.50421142578125
1499 27.635480880737305
1599 22.089792251586914
1699 18.180349349975586
1799 15.423724174499512
1899 13.479543685913086
1999 12.10806